
## Глава 3 Теоремы теории вероятностей

**Кандидат должен знать:**

- Закон больших чисел и усиленный закон больших чисел
- Центральная предельная теорема
- Многомерные распределения

**Материалы:**
- [Райгородский курс по тер. веру лекции 8,9,12,13](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=8)
- [курс ТВ  на степике из частей 2,4](https://stepik.org/course/3089)


## Задачи с кодингом


### 1. Генерация нормальной случайной величины

**Дано:**

Пусть у нас есть случайная величина $\xi \sim U[0,1]$ . 

**Задача:**
1. На основе данной случайной величины с помощью ЦПТ осуществите моделирование  ($n = 10000$) случайной величины  $\eta \sim N(\mu, \sigma^2)$

2. Постройте график полтности распределения и сравните с теоритическим. 

3. Вычислить математического ожидания и дисперсии, сравнить их с истинными значениями.

**Примечания**

1. Для генерации реализаций $\xi$ можно восппользоваться функцией numpy.random.random()
2.  Для построения теоритического графика плотности распределения нормальной величины можно воспользоваться функцией scipy.stats.norm.ppf

### 1. Генерация нормальной случайной величины

Let $X$ be a uniformly distributed random variable, and $\eta$ be a normally distributed random variable.

$\eta = \frac{1}{N_x} \sum_{i=1}^{N_x} X_i$
<br>
$\mu_{\eta} = \frac{1}{N_x} \mu \left( \sum_{i=1}^{N_x} X_i \right) = \frac{1}{N_x} \sum_{i=1}^{N} \mu(X_i) = \mu_X$
<br>

$
\sigma_\eta^2 = \frac{1}{(N_x-1)^2} \sum_{i=1}^{N_x} \sigma_{x_i}^2 = \frac{N_x \cdot \sigma_x^2}{(N_x-1)^2} \approx \left[\text{if } N_x > 100\right] \approx \frac{\sigma_x^2}{N_x}
$


<br><br><br>
$\mu_X = \mu_{\eta}$
<br>
$\sigma_x^2 = N_x \cdot \sigma_\eta^2$

Now I need random uniform values $X$ with $\mu_X$ and $\sigma_x^2$:


\begin{cases}
\mu_X = \frac{a + b}{2} \\
\sigma_x^2 = \frac{(b - a)^2}{12} 
\end{cases}

\begin{cases}
a = \mu_X - \sqrt{3 \cdot \sigma_x^2} \\
b = \mu_X + \sqrt{3 \cdot \sigma_x^2}
\end{cases}


$X = a + Z \cdot (b - a) \quad \text{where} \quad Z \sim U[0, 1]$

In [1]:
import math
from math import comb
from collections import Counter

import numpy as np
from scipy.stats import norm

import plotly.express as px
import plotly.graph_objects as go

In [2]:
def generate_eta_1(E_eta, D_eta, n_x, n_eta):
    mu_x = E_eta
    var_x = n_x * D_eta

    a = mu_x - np.sqrt(3 * var_x)
    b = mu_x + np.sqrt(3 * var_x)

    Z = np.random.random((n_eta, n_x))
    X = a + Z * (b - a)

    return X.mean(axis=1)

### Or another way:

To generate $\eta \sim N(\mu, \sigma^2)$:
1. Generate $X \sim U(0, 1)$ and sum $n_x$ samples to get `sum_uniforms` with the shape of the number of samples $\eta$.
2. Standardize the `sum_uniforms` to get standard normal values. $\mu_X = n_x / 2$, $\text{var}_X = n_x / 12$.
3. Scale and shift using parameters $N(\mu, \sigma^2)$ to get the final normal samples $\eta$.



In [3]:
def generate_eta_2(E_eta, D_eta, n_x, n_eta):
    uniform_random = np.random.random((n_eta, n_x))
    sum_uniforms = np.sum(uniform_random, axis=1)
    standard_normal = (sum_uniforms - n_x/2) / np.sqrt(n_x/12)
    eta = E_eta + np.sqrt(D_eta) * standard_normal
    return eta

### Вычисление мат. ожидания и дисперсии

In [4]:
# choose generation algorithm: generate_eta_1 or generate_eta_2
generate_eta = generate_eta_2

E_eta = -7
D_eta = 13
sigma_eta = np.sqrt(D_eta)
N_x = 1000
N_eta = 10**5


eta = generate_eta(E_eta, D_eta, N_x, N_eta)

print("###  Task 3: Real E and Var VS Estimated E and Var  ###\n")
print(f"Real mu: {E_eta}; Estimated mu: {eta.mean()}")
print(f"Real var: {D_eta}; Estimated var: {eta.var()}")

###  Task 3: Real E and Var VS Estimated E and Var  ###

Real mu: -7; Estimated mu: -6.992836473042595
Real var: 13; Estimated var: 12.985974436714436


In [5]:
fig = px.histogram(
    eta,
    nbins=40,
    title="Distribution of eta",
    labels={"value": "Value"},
    height=700,
    width=500,
)

In [6]:
# fig.show()

![](plots/norm_distribution.png)

### График полтности распределения. Сравние с теоритическим графиком

In [7]:
fig = go.Figure()

norm_x = np.linspace(
    norm.ppf(0.0001, loc=E_eta, scale=sigma_eta), norm.ppf(0.9999, loc=E_eta, scale=sigma_eta), N_eta
)
norm_y = norm.pdf(norm_x, loc=E_eta, scale=sigma_eta)

fig.add_trace(
    go.Scatter(
        x=norm_x,
        y=norm_y,
        mode="lines",
        line=dict(color="darkblue", width=5),
        name="norm pdf",
    )
)

hist_y, hist_x = np.histogram(eta, bins="auto", density=True)
fig.add_trace(
    go.Bar(
        x=hist_x,
        y=hist_y,
        opacity=0.8,
        name="histogram",
        marker=dict(color="darkgreen"),
    )
)

fig = fig.update_layout(
    title="Normal Distribution",
    xaxis_title="Value",
    yaxis_title="Density",
    template="plotly_dark",
    legend=dict(x=0.8, y=1.0),
    height=700,
    width=500,
)

In [8]:
# fig.show()

![](plots/norm_pdf.png)

### 2. Казино 

В казино имеется две игры:

**a. Кубики:**

Игрок платит 4&#36; и бросает два кубика. Если в сумме выпало 8 и более то игрок получает приз равный полученной сумме.

**b. Разгадай код:**

Есть пятизначный код (на каждой позиции стоит цифра от 0 до 9).  Игрок платит 10&#36; и пытается угадать код:
    
    - Если игрок меньше двух позиций то ничего не получает 
    - Если игрок угадал ровно две позиции из 5-ти, то он получает 50
    - Если игроку гадал ровно три позиции из 5-ти, то выигрыш 200
    - Если игроку гадал ровно четыре позиции из 5-ти, то выигрыш 5000
    - Если игрок угадал весь код правильно то выигрыш 100000.
    После каждой попытка код меняется.

**Задача:**

Пусть $\xi ,  \eta $ - случайные величины равные выигрышу(проигрышу) игрока в раунде.
1. Посчитать $E\xi$ и $E\eta$
2. Смоделировать случайные величины $\xi$ и $\eta $
3. Построить графики сходимости среднего выигрыша(проигрыша) игрока к математическому ожиданию.
4. Одинаковая ли скорость сходимости  сл. величины к своему математическому ожиданию. Почему?
4. Какое бы $n$ - количество эеспериментов вы би взяли, для оценки мат. ожидания?




## a. Кубики
### 1) Расчет мат. ожидания

In [9]:
def calc_E_and_D(probs, values):
    E_xi = sum([prob * value for prob, value in zip(probs, values)])
    second_moment = sum([prob * value**2 for prob, value in zip(probs, values)])
    
    D_xi = second_moment - E_xi**2
    return E_xi, D_xi

In [10]:
win_amounts = np.array([0 if i < 8 else i for i in range(0, 13)]) - 4
possible_dice_values = [i + j for i in range(1, 7) for j in range(1, 7)]

count_values = Counter(possible_dice_values)
probs = [count_values[dice_value] / 36 for dice_value in range(0, 13)]
value_to_prob_str = ",  ".join([f"{i}: {count_values[i]}/36" for i in range(0, 13)])

print("Value: Probability -", value_to_prob_str)
print("Probs sum:", sum(count_values.values()) / 36)

Value: Probability - 0: 0/36,  1: 0/36,  2: 1/36,  3: 2/36,  4: 3/36,  5: 4/36,  6: 5/36,  7: 6/36,  8: 5/36,  9: 4/36,  10: 3/36,  11: 2/36,  12: 1/36
Probs sum: 1.0


In [11]:
E_xi, D_xi = calc_E_and_D(probs, win_amounts)

print("Theoretical E_xi:", round(E_xi, 5))
print("Theoretical D_xi:", round(D_xi, 5))

Theoretical E_xi: -0.11111
Theoretical D_xi: 21.82099


### 2) Моделирование случайной величины

In [12]:
def generate_xi(n_samples, win_amounts):
    dice_values = np.random.randint(1, 7, 2 * n_samples)
    dice_sums = dice_values[::2] + dice_values[1::2]
    xi = win_amounts[dice_sums]
    return xi

In [13]:
np.random.seed(48)
n_samples = 9 * 10**3

xi_values = generate_xi(n_samples, win_amounts)

print("Theoretical E_xi:", round(E_xi, 5))
print("Simulated E_xi:", xi_values.mean())

Theoretical E_xi: -0.11111
Simulated E_xi: -0.11477777777777778


### 3. Сходимость среднего выигрыша к математическому ожиданию

In [14]:
def plot(values, mean_value):
    cumulative_means = np.cumsum(values) / np.arange(1, len(values) + 1)

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=np.arange(1, len(values) + 1),
            y=cumulative_means,
            mode="lines",
            name="Среднее значение выигрыша",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=[1, len(values)],
            y=[mean_value, mean_value],
            mode="lines",
            line=dict(dash="dash", color="red"),
            name="Математическое ожидание",
        )
    )

    fig.update_layout(
        title="Сходимость среднего выигрыша к математическому ожиданию",
        xaxis_title="Количество наблюдений",
        yaxis_title="Средний выигрыш",
        legend=dict(x=0.8, y=1.0),
        template="plotly_dark",
    )

    fig.show()

In [15]:
# plot(xi_values, E_xi)

![](plots/dice_expectation.png)

### 5) Оценка количества экспериментов

1. Согласно ЦПТ, выборочное среднее $\bar{X}_n$ выборки размера $n$ из генеральной совокупности с математическим ожиданием $\mu$ и дисперсией $\sigma^2$ имеет нормальное распределение:
   $$\bar{X}_n \sim N\left(\mu, \frac{\sigma^2}{n}\right)$$

2. Чтобы вероятность того, что выборочное среднее $\bar{X}_n$ попадает в интервал [$\mu - \epsilon$, $\mu + \epsilon$] была равна $p$:
   $$P\left(|\bar{X}_n - \mu| < \epsilon\right) = p$$

3. Стандартизируем: вычитанием математического ожидания и делением на стандартное отклонение:
   $$P\left(|\bar{X}_n - \mu| < \epsilon\right) = P\left(\left| \frac{\bar{X}_n - \mu}{\frac{\sigma}{\sqrt{n}}} \right| < \frac{\epsilon}{\frac{\sigma}{\sqrt{n}}}\right) = P\left(|Z| < \frac{\epsilon \sqrt{n}}{\sigma}\right) = p$$

4. Для p = 0.95:
   $$\frac{\epsilon \sqrt{n}}{\sigma} = z = 1.96$$ 

5. Находим минимальное количество экспериментов $n$, необходимое для достижения указанной точности:
   $$n \geq \left( \frac{z \sigma}{\epsilon} \right)^2$$


In [16]:
def estimate_n(variance_x, eps_x, x):
    return ((x * np.sqrt(variance_x)) / eps_x) ** 2

In [17]:
z = 1.96
eps = 0.05

n = int(estimate_n(D_xi, eps, z))

print("For 95% confidence level, and 5% error, n should be at least", n)

For 95% confidence level, and 5% error, n should be at least 33531


## b. Разгадай код
### 1)  Расчет мат. ожидания

In [18]:
p = 0.1
q = 0.9
guess_code_win_amounts = np.array([0, 0, 50, 200, 5000, 100000]) - 10
code_game_probs = [comb(5, i) * p**i * q ** (5 - i) for i in range(6)]

print("Probs", list(map(lambda x: round(x, 5), code_game_probs)))
print("Probs sum", sum(code_game_probs))

Probs [0.59049, 0.32805, 0.0729, 0.0081, 0.00045, 1e-05]
Probs sum 1.0


In [19]:
E_eta, D_eta = calc_E_and_D(code_game_probs, guess_code_win_amounts)

print("Theoretical E_eta:", round(E_eta, 5))
print("Theoretical D_eta:", round(D_eta, 5))

Theoretical E_eta: -1.485
Theoretical D_eta: 111683.74478


### 2) Моделирование случайной величины

In [20]:
def generate_eta(n_samples, guess_code_win_amounts):
    random_values_code = np.random.randint(0, 10, (n_samples, 5))
    code_guesses = np.random.randint(0, 10, (n_samples, 5))
    
    is_guess_random_code = code_guesses == random_values_code
    random_guesses = is_guess_random_code.sum(axis=1)
    eta = guess_code_win_amounts[random_guesses]
    return eta

In [21]:
np.random.seed(12)
N_plays = 1 * 10**6

eta_values = generate_eta(N_plays, guess_code_win_amounts)

print("Theoretical E_eta:", round(E_eta, 5))
print("Simulated E_eta", eta_values.mean())

Theoretical E_eta: -1.485
Simulated E_eta -1.4936


### 3. Сходимость среднего выигрыша к математическому ожиданию

In [22]:
# plot(random_guesses_win_amounts, E_eta)

![](plots/code_expectation.png)

### 4) Сравнение скорости сходимости

Скорости сходимости разные, так как есть значительные различия в дисперсиях случайных величин.


In [23]:
print("D_xi:", D_xi)
print("D_eta:", D_eta)

D_xi: 21.82098765432099
D_eta: 111683.74477500003


### 5) Оценка количества экспериментов

In [24]:
n = int(estimate_n(D_eta, eps, z))

print("For 95% confidence level, and 5% error, n should be at least", n)

For 95% confidence level, and 5% error, n should be at least 171617709


### Случайный граф


**Условие**

Пусть G(n,p) -  случайный граф на $n$ вершинах, где $p$ - это вероятность появления ребра между любыми двумя вершинами.

Обозначим за $\xi(n,p)$ - количество треугольников (т.е. полных подграфов на трёх вершинах) в случайном графе  

**Задача 1:**

1. Найти $E\xi$
2. Реализовать моделирование случайного случайного графа и функцию нахождения количества треугольников. 
3. Рассмотрим G(3,p) - случайный граф на трёх вершинах. Реализуйте $\tau$ - случайная величина равная 1 если граф полный и 0 иначе.


**Задача 2:**

- Пусть M(k) - среднее количество треугольников после k генераций случайного графа. 
- И пусть T(k)  - среднее количество треугольников после генераций случайного графа на трёх вершинах (т.е. на каждой итерации генерируем $C_n^3$ графов на трёх вершинах).

1. Верно ли что $M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ 
2. Верно ли что $T(k) \rightarrow  E\xi$ при $k \rightarrow \infty$

3. Рассмотрите два графа $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.  Построите графики  $M(k)$, $T(k)$ и $E\xi$ (константа) для заданных графов.

**Задача 3**:

Проделайте аналогичные рассуждения для $D \xi$ т.е.
- Найдите  $D \xi$  математически.
- Посчитайте дисперсию количества треугольников, k раз реализуя случайный граф.
- Посчитайте дисперсию количества треугольников, k раз генерируя  $C_n^3$ реализаций случайной величины $\tau$ .
- оцените сходимость
- Постройте графики для графов  $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$.

## Задача 1
#### Случайный граф на n вершинах. Найти $E\xi$

Пусть I - случайная величина равная 1 если между конкретными тремя вершинами образовался треугольник. Тогда $\xi = \sum_{i=1}^{C_n^3} I_i$.

$E\xi = E\sum_{i=1}^{C_n^3} I_i = \sum_{i=1}^{C_n^3} E I_i = C_n^3 \cdot p^3$

#### G(3,p) - случайный граф на трех вершинах.

Для одного ргафа: $E\tau = 1 \cdot p^3 + 0 \cdot (1 - p^3) = p^3$

#### Моделирования случайного графа и реализация функции нахождения количества треугольников.

В качестве случайного графа буду использовать матрицу смежности, заполненную только выше главной диагонали.

Квадрат матрицы смежности позволяет подсчитать количество путей длиной 2 между вершинами. Элементы в квадрате матрицы смежности могут принимать значения от 0 до $n - 1$, где $n$ — количество вершин в графе. Поэтому, для эффективного использования памяти, можно использовать тип данных int8 при $n \leq 128$, иначе int16 при $128 < n < 32768$.

Последующее поэлементное умножение матрицы позволит найти количество треугольников в графе. Поскольку я использую только верхнетреугольную матрицу смежности, повторы при подсчете треугольников не появляются.


In [25]:
def generate_graph(n_graphs, n_nodes, edge_p):
    dtype = np.int8 if n_nodes <= 128 else np.int16
    random_values = (
            np.random.random((n_graphs, n_nodes, n_nodes)) < edge_p
    ).astype(dtype)
    
    adjacency_matrices = np.triu(random_values, k=1)
    return adjacency_matrices


def count_triangles(graphs):
    squared_matrix = np.matmul(graphs, graphs)
    triangles_matrix = graphs * squared_matrix
    n_triangles = np.sum(np.triu(triangles_matrix, k=1), axis=(1, 2))
    return n_triangles

In [26]:
%%time
random_graphs = generate_graph(1, 300, 1)
triangles = count_triangles(random_graphs)

print("mean triangles", np.mean(triangles))

mean triangles 4455100.0
CPU times: user 28.8 ms, sys: 157 µs, total: 28.9 ms
Wall time: 28.1 ms


## Задача 2

In [27]:
k = 10**5
n = 10
p = 0.5

print("E_xi = E_tau =", math.comb(n, 3) * p**3)

E_xi = E_tau = 15.0


#### Случайный граф на n вершинах. M(k):

In [28]:
%%time
random_graphs = generate_graph(k, n, p)
n_triangles_in_each_graph = count_triangles(random_graphs)

CPU times: user 145 ms, sys: 24.3 ms, total: 170 ms
Wall time: 168 ms


In [29]:
M_k = np.mean(n_triangles_in_each_graph)
print("M_k:", M_k)

M_k: 15.0068


#### G(3,p) - случайный граф на трех вершинах. T(k):

In [30]:
random_graphs = (np.random.random((k, math.comb(n, 3))) < p**3).astype(np.int8)
N_triangles = random_graphs.sum(axis=1)

In [31]:
T_k = np.mean(N_triangles)
print("T_k:", T_k)

T_k: 15.01026


$M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ <br>
$T(k) \rightarrow E\xi$ при $k \rightarrow \infty$

Оба утверждения верны

Рассмотрим $G_1(10, \frac{1}{2})$ и $G_2(10, \frac{1}{20})$. И построим графики  $M(k)$, $T(k)$ и $E\xi$


In [32]:
def get_Mk_and_Tk_and_Exi(n_graphs, n_nodes, edge_p):
    E_xi = math.comb(n_nodes, 3) * edge_p**3

    g_1 = generate_graph(n_graphs, n_nodes, edge_p)
    n_triangles_1 = count_triangles(g_1)

    g_2 = (np.random.random((n_graphs, math.comb(n_nodes, 3))) < edge_p**3).astype(
        np.int8
    )
    n_triangles_2 = g_2.sum(axis=1)

    M_k = np.cumsum(n_triangles_1) / np.arange(1, n_graphs + 1)
    T_k = np.cumsum(n_triangles_2) / np.arange(1, n_graphs + 1)

    return M_k, T_k, E_xi


def make_plot(values, names, const_val, const_name, title, val_length):
    fig = go.Figure()

    for value, name in zip(values, names):
        fig.add_trace(
            go.Scatter(
                x=np.arange(1, val_length + 1),
                y=value,
                mode="lines",
                line=dict(width=3),
                name=name,
            )
        )

    fig.add_trace(
        go.Scatter(
            x=[1, val_length],
            y=[const_val, const_val],
            mode="lines",
            line=dict(dash="dash", color="darkorange", width=3),
            name=const_name,
        )
    )

    fig.update_layout(
        title=title,
        xaxis_title="Number of graphs",
        yaxis_title="Value",
        legend=dict(x=0.94, y=1.0),
        template="plotly_dark",
    )

    fig.show()

In [33]:
np.random.seed(43)
k = 10**3
n_nodes = 10

p_1 = 0.5

M_k, T_k, E_xi = get_Mk_and_Tk_and_Exi(k, n_nodes, p_1)
print("E_xi:", E_xi)
print("M_k:", M_k.mean())
print("T_k:", T_k.mean())

E_xi: 15.0
M_k: 14.60359209012895
T_k: 14.916768929094793


In [34]:
title = f"Convergence of M(k) and T(k) to E_xi for G({n_nodes}, {p_1})"
values = [M_k, T_k]
names = ["M(k)", "T(k)"]

# make_plot(values, names, E_xi, "E_xi", title, k)

![](plots/random_graph/E_convergence_G_10_05.png)

In [35]:
np.random.seed(41)
p_2 = 0.05

M_k, T_k, E_xi = get_Mk_and_Tk_and_Exi(k, n_nodes, p_2)
print("E_xi:", E_xi)
print("M_k:", M_k.mean())
print("T_k:", T_k.mean())

E_xi: 0.015000000000000003
M_k: 0.012020277862335947
T_k: 0.02063560314747795


In [36]:
title = f"Convergence of M(k) and T(k) to E_xi for G({n_nodes}, {p_2})"
values = [M_k, T_k]
names = ["M(k)", "T(k)"]

# make_plot(values, names, E_xi, "E_xi", title, k)

![](plots/random_graph/E_convergence_G_10_005.png)

## Задача 3

### Относительно просто посчитать дисперсию для графа на трёх вершинах:

$D \tau = E I^2 - (E I)^2 = C_n^3 \cdot p^3 - (C_n^3 \cdot p^3)^2;$   где I = 1  если образовался треугольник и 0 иначе
<br><br>

### Для случайного графа на n вершинах:

$D \xi = E \xi^2 - (E \xi)^2$
<br>
$E \xi^2 = E \left( \sum_{i=1}^{C_n^3} I_i \right)^2 = 
\sum_{i=1}^{C_n^3} E I_i^2 + \sum_{i \neq j} E I_i I_j = 
C_n^3 \cdot p^3 + C_n^3 \cdot C_{n-3}^3 \cdot p^6 + 3 \cdot C_n^3 \cdot C_{n-3}^2 \cdot p^6 + 3 \cdot C_n^3 \cdot (n-3) \cdot p^5$
<br>
$(E \xi)^2 = (C_n^3 \cdot p^3)^2$

In [37]:
def get_theoretical_xi_and_tau(n, p):
    tau_var = comb(n, 3) * (p**3 - p**6)

    xi_squared_E = (comb(n, 3) * p**3) ** 2
    xi_second_moment = (
        comb(n, 3) * p**3
        + comb(n, 3) * comb(n - 3, 3) * p**6
        + 3 * comb(n, 3) * comb(n - 3, 2) * p**6
        + 3 * comb(n, 3) * (n - 3) * p**5
    )
    xi_var = xi_second_moment - xi_squared_E

    return xi_var, tau_var


def generate_graphs_and_get_n_triangles(n_graphs, n_nodes, edge_p):
    xi_graphs = generate_graph(n_graphs, n_nodes, edge_p)
    xi_triangles = count_triangles(xi_graphs)

    tau_graphs = (np.random.random((n_graphs, math.comb(n_nodes, 3))) < p**3).astype(
        np.int8
    )
    tau_triangles = tau_graphs.sum(axis=1)

    return xi_triangles, tau_triangles


def get_vars(n_graphs, n_nodes, edge_p):
    xi_triangles, tau_triangles = generate_graphs_and_get_n_triangles(
        n_graphs, n_nodes, edge_p
    )
    xi_var = xi_triangles.var()
    tau_var = tau_triangles.var()

    theoretical_xi_var, theoretical_tau_var = get_theoretical_xi_and_tau(
        n_nodes, edge_p
    )

    return xi_var, tau_var, theoretical_xi_var, theoretical_tau_var

In [38]:
np.random.seed(42)

p = 0.5
k = 2 * 10**5
n = 10

xi_var, tau_var, theoretical_xi_var, theoretical_tau_var = get_vars(k, n, p)

print("### G(10, 0.5) ###\n")
print(f"  Theoretical Variances: \nxi: {theoretical_xi_var} \ntau: {theoretical_tau_var}")
print(f"\n  Real Variances: \nxi: {xi_var} \ntau: {tau_var}")

### G(10, 0.5) ###

  Theoretical Variances: 
xi: 52.5 
tau: 13.125

  Real Variances: 
xi: 52.6366167936 
tau: 13.191531138775002


In [39]:
p = 0.05

xi_var, tau_var, theoretical_xi_var, theoretical_tau_var = get_vars(k, n, p)

print("### G(10, 0.05) ###\n")
print(f"  Theoretical Variances: \nxi: {theoretical_xi_var} \ntau: {theoretical_tau_var}")
print(f"\n  Real Variances: \nxi: {xi_var} \ntau: {tau_var}")

### G(10, 0.05) ###

  Theoretical Variances: 
xi: 0.015746250000000003 
tau: 0.014998125000000005

  Real Variances: 
xi: 0.015945931775 
tau: 0.014932244374999997


#### Построим графики для G(10, 0.5)

In [40]:
k = 10**4
n = 10

In [41]:
np.random.seed(13)
p = 0.5

xi_triangles, tau_triangles = generate_graphs_and_get_n_triangles(k, n, p)
theoretical_xi_var, theoretical_tau_var = get_theoretical_xi_and_tau(n, p)

xi_vals = [np.var(xi_triangles[:i]) for i in range(10, k)]
tau_vals = [np.var(tau_triangles[:i]) for i in range(10, k)]

In [42]:
xi_title = f"xi Convergence of Variances for G({n}, {p})"
tau_title = f"tau Convergence of Variances for G({n}, {p})"

# make_plot([xi_vals], ["xi"], theoretical_xi_var, "Theoretical xi", xi_title, val_length=len(xi_vals))
# make_plot([tau_vals], ["tau"], theoretical_tau_var, "Theoretical tau", tau_title, val_length=len(tau_vals))

![](plots/random_graph/xi_var_convergence_G_10_05.png)

![](plots/random_graph/tau_var_convergence_G_10_05.png)

#### И для G(10, 0.05)

In [43]:
np.random.seed(42)
p = 0.05

xi_triangles, tau_triangles = generate_graphs_and_get_n_triangles(k, n, p)
theoretical_xi_var, theoretical_tau_var = get_theoretical_xi_and_tau(n, p)

xi_vals = [np.var(xi_triangles[:i]) for i in range(10, k)]
tau_vals = [np.var(tau_triangles[:i]) for i in range(10, k)]

In [44]:
xi_title = f"xi Convergence of Variances for G({n}, {p})"
tau_title = f"tau Convergence of Variances for G({n}, {p})"

# make_plot([xi_vals], ["xi"], theoretical_xi_var, "Theoretical xi", xi_title, val_length=len(xi_vals))
# make_plot([tau_vals], ["tau"], theoretical_tau_var, "Theoretical tau", tau_title, val_length=len(tau_vals))

![](plots/random_graph/xi_var_convergence_G_10_005.png)

![](plots/random_graph/tau_var_convergence_G_10_005.png)
